In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../data'
group_dir_ = os.path.join(dir_, 'groups/low')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

1053

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in tqdm(df['tid'].unique()):
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
70,70,3.651472
5521,5521,2.635328
390,390,2.554606
2519,2519,2.535613
83,83,2.530864
13496,13496,2.521368
210,210,2.431149
5716,5716,2.426401
1036,1036,2.302944
1464,1464,2.288699


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [8]:
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 20
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
for NOV in NOVs:
    print(NOV)
print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
for COV in COVs:
    print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.523836657170008, 0.5238366571700005, 0.5238366571699956, 0.5238366571699948, 0.5238366571699942, 0.5238366571699938, 0.5238366571699933]
[0.5257897893632204, 0.5248851738369719, 0.5245621249057782, 0.5243201243201314, 0.5242382874327381, 0.5240497236223782, 0.5240097417516804]
[0.5322420047720411, 0.528558117866312, 0.5274258133364768, 0.5265144320699952, 0.5261079455523974, 0.52591375070008, 0.5255138927181972]
[0.542331499764581, 0.5365962479170179, 0.5327444415073352, 0.5305980028202313, 0.5302310857866482, 0.52852290159983, 0.5282143185369017]
[0.5590987891080303, 0.5489120631678424, 0.5445413692836457, 0.5384464301131036, 0.536579020259581, 0.5344668468600128, 0.5323882608828846]
[0.5761372993136591, 0.5645708278832032, 0.5589043577016166, 0.5537540648651824, 0.547062955049071, 0.5457812842428253, 0.5413855956866725]
[0.5767862826957906, 0.5901458546112609, 0.5797064841738435, 0.5705732539065943, 0.5684641104780098, 0.5651691138870727, 0.5598214012192562]
[0.5878787161335564, 